In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import bisect
sys.path.insert(-1, os.path.dirname(os.path.dirname(os.getcwd())))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datasets import chalearn2014
from sltools.preprocess import fix_contrast
from sltools.transform import transform_frames, Transformation

In [ ]:
matplotlib.rc('axes', axisbelow=True, linewidth=1.4)

In [ ]:
fig = plt.figure(figsize=(12, 17), dpi=300)

cnames = [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo']

for i in range(1, 21):
    for seq in chalearn2014.default_splits()[0]:
        glosses = chalearn2014.glosses(seq)
        if i in glosses[:, 0]:
            _, start, stop = glosses[glosses[:, 0] == i][0]
            
            t = (start + stop) // 2
            frame = chalearn2014.bgr_frames(seq)[t]
            pose = chalearn2014.positions(seq)[t]
            x1, x2, y1, y2 = np.min(pose[:, 0]), np.max(pose[:, 0]), np.min(pose[:, 1]), np.max(pose[:, 1])
            x1, x2, y1, y2 = max(0, x1 - 30), min(640, x2 + 30), max(0, y1 - 30), min(480, y2 + 30)
            ax = fig.add_subplot(4, 5, i)
            ax.imshow(frame[y1:y2, x1:x2, ::-1])  # fix bgr to RGB
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_xlabel(cnames[i], fontsize=18)
            
            break

fig.tight_layout(pad=-10, w_pad=-3, h_pad=1)

In [ ]:
seq = 532
g = 1

glosses = chalearn2014.glosses(seq)
frames = chalearn2014.bgr_frames(seq)
poses = chalearn2014.positions(seq)

print(glosses[:, 0])

joints = chalearn2014.JointType
important_joints = [joints.HandLeft, joints.HandRight,
                    joints.ElbowLeft, joints.ElbowRight,
                    joints.ShoulderLeft, joints.ShoulderRight,
                    joints.Head]
x1, x2, y1, y2 = (np.min(poses[:, important_joints, 0]), np.max(poses[:, important_joints, 0]), 
                  np.min(poses[:, important_joints, 1]), np.max(poses[:, important_joints, 1]))
x1, x2, y1, y2 = max(0, x1 - 30), min(640, x2 + 30), max(0, y1 - 30), min(480, y2 + 30)

fig = plt.figure(figsize=(18, 12))
timesteps = np.linspace(glosses[g, 1], glosses[g, 2], 6).astype(np.int)
timesteps = np.linspace(1500, 1550, 6).astype(np.int)
for p, t in enumerate(timesteps):
    ax = fig.add_subplot(2, 3, p + 1)
    ax.imshow(frames[t][:, :, ::-1])  # fix bgr to RGB
    ax.scatter(poses[t, important_joints, 0], poses[t, important_joints, 1], marker='+', s=400, color='r', linewidth=3)
    ax.axis('off')


In [ ]:
frames = chalearn2014.bgr_frames(0)
fig = plt.figure(figsize=(18, 12))
fig.set_facecolor('none')
for p, i in enumerate(range(363, 367, 1)):
    ax = fig.add_subplot(1, 5, p + 1)
    ax.imshow(frames[i][100:280, 220:300][:, :, ::-1])  # fix bgr to RGB
    ax.axis('off')

plt.gca().set_facecolor('none')
# fig.tight_layout(h_pad=0, w_pad=0, pad=0)

In [ ]:
j = chalearn2014.JointType

In [ ]:
fig = plt.figure(figsize=(11, 6))
fig.set_facecolor('none')

ax = fig.add_subplot(1, 3, 1)
ax.imshow(chalearn2014.bgr_frames(0)[367][0:350, 180:400][:, :, ::-1])  # fix bgr to RGB
ax.axis('off')
ax.set_facecolor('none')

zmap = chalearn2014.z_maps(0)[367][0:350, 180:400]
zmap[zmap<1200] = np.max(zmap)
zmap[zmap > 3000] = 3000
ax = fig.add_subplot(1, 3, 2)
ax.imshow(zmap, cmap="gray")
ax.axis('off')
ax.set_facecolor('none')

fig.set_facecolor('none')
ax = fig.add_subplot(1, 3, 3)
j = chalearn2014.JointType
pose = chalearn2014.positions(0)[367]
paths = [[j.WristLeft, j.ElbowLeft, j.ShoulderLeft, j.ShoulderRight, j.ElbowRight, j.WristRight],
         [j.KneeLeft, j.HipLeft, j.HipCenter, j.HipRight, j.KneeRight],
         [j.HipCenter, j.ShoulderCenter, j.Head]]
for path in paths:
    ax.plot(pose[path, 0], pose[path, 1], color="blue", linewidth=3)
    ax.scatter(pose[path, 0], pose[path, 1], color="red", s=100, alpha=1)
ax.plot()
ax.set_ylim([0, 350])
ax.set_xlim([180, 400])
ax.invert_yaxis()
ax.axis('off')
ax.set_facecolor('none')
ax.set_aspect("equal")

fig.tight_layout()

fig.savefig("/home/granger/chalearn_sample.pdf", bbox_inches="tight")

In [ ]:
frames = chalearn2014.z_maps(0)
fig = plt.figure(figsize=(18, 10))
fig.set_facecolor('none')
for p, i in enumerate(range(105, 140, 5)):
    fig.add_subplot(1, 7, p + 1)
    z = np.clip(frames[i][:, 150:450], 2200, 2700)
    z = (z - 2200) / 500
    z = np.repeat(z[:, :, None], 4, 2)
    z[:, :, 3] = 1 - z[:, :, 3] * .2
    plt.imshow(z, cmap='gray')
    plt.axis('off')
    plt.gca().set_facecolor('none')
    
# fig.tight_layout(h_pad=0, w_pad=0, pad=0)

In [ ]:
skeleton2d = chalearn2014.positions(63)[105:140:5]
skeleton3d = chalearn2014.positions_3d(63)[105:140:5]

paths = [[
    chalearn2014.JointType.FootLeft,
    chalearn2014.JointType.AnkleLeft,
    chalearn2014.JointType.KneeLeft,
    chalearn2014.JointType.HipLeft,
    chalearn2014.JointType.HipCenter,
    chalearn2014.JointType.Spine,
    ], [
    chalearn2014.JointType.FootRight,
    chalearn2014.JointType.AnkleRight,
    chalearn2014.JointType.KneeRight,
    chalearn2014.JointType.HipRight,
    chalearn2014.JointType.HipCenter,
    chalearn2014.JointType.Spine
    ], [
    chalearn2014.JointType.Spine,
    chalearn2014.JointType.ShoulderLeft,
    chalearn2014.JointType.ElbowLeft,
    chalearn2014.JointType.WristLeft,
    chalearn2014.JointType.HandLeft
    ], [
    chalearn2014.JointType.Spine,
    chalearn2014.JointType.ShoulderRight,
    chalearn2014.JointType.ElbowRight,
    chalearn2014.JointType.WristRight,
    chalearn2014.JointType.HandRight
    ], [
    chalearn2014.JointType.Head,
]]

fig = plt.figure(figsize=(18, 10))
fig.set_facecolor('none')
for i, _ in enumerate(range(105, 140, 5)):
    fig.add_subplot(1, 7, i + 1)
    plt.xlim(150, 450)
    plt.ylim(480, 0)
    for p in paths:
        plt.scatter(x=skeleton2d[i, p, 0], y=skeleton2d[i, p, 1], s=1 / (skeleton3d[i, p, 2] + 0.1) * 50)
        plt.plot(skeleton2d[i, p, 0], skeleton2d[i, p, 1])
    plt.axis('off')
    plt.gca().set_aspect('equal')

plt.gca().set_facecolor('none')
plt.show()
# fig.tight_layout(h_pad=0, w_pad=0, pad=0)

In [ ]:
plt.figure(figsize=(7, 7))
plt.imshow(chalearn2014.bgr_frames(17)[10][80:380, 240:420, ::-1])
plt.gca().axis('off')

In [ ]:
for s in range(3):
    print(np.sum([len(chalearn2014.glosses(r)) for r in chalearn2014.default_splits()[s]]))

In [ ]:
print(np.sum([len(chalearn2014.glosses(r)) for r in chalearn2014.all_recordings()]))

In [ ]:
g = np.concatenate([chalearn2014.glosses(r) for r in chalearn2014.default_splits()[0]])
h, e = np.histogram(g[:, 2] - g[:, 1], 25)
h = h / h.sum() * 100
m = np.mean(g[:, 2] - g[:, 1])
mb = bisect.bisect(e, m)

plt.figure(figsize=(5, 5), dpi=150)
plt.bar(e[:-1], h, width=e[1:] - e[:-1], edgecolor='black', linewidth=1)
plt.bar([m], [h[mb]], width=1, color='darkred')

xticks = np.arange(0, 120, 20, dtype=np.float)
xticks[bisect.bisect(xticks, m)] = m
plt.yticks(np.arange(0, 20, 4))
plt.xticks(xticks)
plt.gca().set_xticklabels(["{:.4g}".format(t) for t in xticks])
plt.xlabel('duration in frames')
plt.ylabel('% of instances')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().grid(axis='y', color='gray', linestyle=':', linewidth=1, which='major', zorder=100)

plt.savefig("/home/granger/duration_dist.pdf", bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(8, 6))

im1 = chalearn2014.bgr_frames(0)[380]
pose1 = chalearn2014.positions(0)[380]
pose_ref2d = pose1[j.ShoulderCenter]
pose_ref3d = chalearn2014.positions_3d(0)[380][j.ShoulderCenter]

ax = fig.add_subplot(2, 3, 1)
ax.imshow(im1[0:300, 150:450, ::-1], cmap='gray', clim=(0, 1))
ax.axis('off')
ax.set_title("original", fontsize=18)

im2 = fix_contrast(im1[None, ...])[0]

ax = fig.add_subplot(2, 3, 2)
ax.imshow(im2[0:300, 150:450], cmap='gray', clim=(0, 1))
ax.axis('off')
ax.set_title("neutral colors", fontsize=18)

t = Transformation(ref2d=pose_ref2d, ref3d=pose_ref3d, frame_width=640, zshift=4 - pose_ref3d[2])

im3 = transform_frames(im2[None, ...], t)[0]
ax = fig.add_subplot(2, 3, 3)
ax.imshow(im3[0:300, 150:450], cmap="gray", clim=(0, 1))
ax.axis('off')
ax.set_title("apparent distance shift", fontsize=18)

t = Transformation(ref2d=pose_ref2d, ref3d=pose_ref3d, frame_width=640, tilt=7 * np.pi / 180)

im4 = transform_frames(im2[None, ...], t)[0]
ax = fig.add_subplot(2, 3, 4)
ax.imshow(im4[0:300, 150:450], cmap="gray", clim=(0, 1))
ax.axis('off')
ax.set_title("tilt", fontsize=18)

t = Transformation(ref2d=pose_ref2d, ref3d=pose_ref3d, frame_width=640, xscale=.7, yscale=1.3)

im5 = transform_frames(im2[None, ...], t)[0]
ax = fig.add_subplot(2, 3, 5)
ax.imshow(im5[0:300, 150:450], cmap='gray', clim=(0, 1))
ax.axis('off')
ax.set_title("xyz-scale", fontsize=18)

fig.tight_layout(pad=-10, w_pad=-3, h_pad=-7)
fig.savefig("/home/granger/transformations.pdf", bbox_inches="tight")